In [3]:
import numpy as np
import pandas as pd
import torch
from scipy.special import softmax
from datasets import load_dataset, load_metric
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from transformers import pipeline
import emoji

In [4]:
tokenizer = RobertaTokenizer.from_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')
model = RobertaForSequenceClassification.from_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')

tokenizer.save_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')
model.save_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def preprocess(texts):

  # remove URLs
  texts = re.sub(r'https?://\S+', "", texts)
  texts = re.sub(r'www.\S+', "", texts)
  # remove '
  texts = texts.replace('&#39;', "'")
  # remove symbol names
  texts = re.sub(r'(\#)(\S+)', r'hashtag_\2', texts)
  texts = re.sub(r'(\$)([A-Za-z]+)', r'cashtag_\2', texts)
  # remove usernames
  texts = re.sub(r'(\@)(\S+)', r'mention_\2', texts)
  # demojize
  texts = emoji.demojize(texts, delimiters=("", " "))

  return texts.strip()

In [5]:
df = pd.read_csv('fs.csv')
texts = list(df.text.values)
labels = list(df.label.values)
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.2)

In [6]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

In [7]:
class myds(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = myds(train_encodings, train_labels)
test_dataset = myds(test_encodings, test_labels)

In [8]:
def cm(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, -1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="steps")
metric = load_metric("accuracy")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=cm,
    tokenizer = tokenizer
)

In [10]:
trainer.train()

***** Running training *****
  Num examples = 8000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3000


Step,Training Loss,Validation Loss,Accuracy
500,0.352000,0.341263,0.913500
1000,0.315900,0.282893,0.923000
1500,0.164400,0.322900,0.930000
2000,0.156700,0.287294,0.938000
2500,0.070700,0.334161,0.940500
3000,0.048600,0.359750,0.940000


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_tra

TrainOutput(global_step=3000, training_loss=0.1847122739156087, metrics={'train_runtime': 311.3207, 'train_samples_per_second': 77.091, 'train_steps_per_second': 9.636, 'total_flos': 1541666340000000.0, 'train_loss': 0.1847122739156087, 'epoch': 3.0})

In [19]:
tokenizer = RobertaTokenizer.from_pretrained('test_trainer/checkpoint-3000/')
model = RobertaForSequenceClassification.from_pretrained('test_trainer/checkpoint-3000/')

Didn't find file test_trainer/checkpoint-3000/added_tokens.json. We won't load it.
Didn't find file test_trainer/checkpoint-3000/tokenizer.json. We won't load it.
loading file test_trainer/checkpoint-3000/vocab.json
loading file test_trainer/checkpoint-3000/merges.txt
loading file None
loading file test_trainer/checkpoint-3000/special_tokens_map.json
loading file test_trainer/checkpoint-3000/tokenizer_config.json
loading file None
loading configuration file test_trainer/checkpoint-3000/config.json
Model config RobertaConfig {
  "_name_or_path": "zhayunduo/roberta-base-stocktwits-finetuned",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "n

In [12]:
d = pd.read_json('testing.json')

In [21]:
nlp = pipeline("text-classification", model=model, tokenizer=tokenizer)


In [24]:
d["lb"] = [nlp(i) for i in d.text]

In [32]:
d['label'] = [i[0]['label'] for i in d.lb]

In [34]:
d['sentiment'] = (d.label == 'LABEL_1').astype(int)

In [36]:
d.to_csv('sentimentdone.csv', index = False)